<a href="https://colab.research.google.com/github/sergionl/TP_complejidad/blob/main/TravellingSalesman.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Fuerza Bruta


In [102]:
!git clone https://github.com/lmcanavals/algorithmic_complexity.git

fatal: destination path 'algorithmic_complexity' already exists and is not an empty directory.


## Imports

In [103]:
from itertools import permutations
import pandas as pd
import numpy as np
import networkx as nx
import math
import json
import sys

In [104]:
from algorithmic_complexity.aclib import graphstuff as gs

## Codigo DataSet

In [105]:
url="https://raw.githubusercontent.com/lmcanavals/algorithmic_complexity/main/data/poblaciones.csv"
poblacionesDF = pd.read_csv(url)

In [106]:
print(len(poblacionesDF))
poblacionesDF.head()

3753


,DEPARTAMENTO,PROVINCIA,DISTRITO,CENTRO POBLADO,LATITUD,LONGITUD,NMI1,NMI2
0,AMAZONAS,BAGUA,ARAMANGO,EL MUYO (C.P.L.),-5.422545,-78.452990,311,1206
1,AMAZONAS,BAGUA,ARAMANGO,RESERVA UJUCANO,-5.427565,-78.444372,44,129
2,AMAZONAS,BAGUA,ARAMANGO,TUTUMBEROS,-5.327317,-78.444720,49,179
3,AMAZONAS,BAGUA,BAGUA,ALENYA BAJO,-5.682439,-78.495229,3,8
4,AMAZONAS,BONGARA,CUISPES,MITOPAMPA,-5.938560,-77.965302,1,4


In [164]:
poblacionesDF.loc[:,'order']=0
id = list(range(len(poblacionesDF)))
poblacionesDF.loc[:,'ID']=id
poblacionesDF.loc[:,'anterior']=None
poblacionesDF.loc[:,'siguiente']=None
poblacionesDF.loc[:,'representante']=None
poblacionesDF.loc[:,'representante1']=None
nomprovincias = poblacionesDF['PROVINCIA'].unique()
#print(len(nomprovincias))
#nomprovincias


In [ ]:
provincias = dict()
for nom in nomprovincias:
  provincias[nom] = poblacionesDF[poblacionesDF['PROVINCIA'] == nom]
  print(nom, len(provincias[nom]))

In [109]:
#@title Provincia 
#@markdown Ingrese la provincia

no_type_checking = ''
nom_provincia = 'BAGUA' #@param {type:"string"}
nom_distrito = 'ARAMANGO' #@param {type:"string"}



In [110]:
nomdistritos = provincias[nom_provincia]['DISTRITO'].unique()
print(len(nomdistritos))
nomdistritos

2


array(['ARAMANGO', 'BAGUA'], dtype=object)

In [111]:
distritos = dict()
for nom in nomdistritos:
  distritos[nom] = provincias[nom_provincia][provincias[nom_provincia]['DISTRITO'] == nom]
  print(nom, len(distritos[nom]))

ARAMANGO 3
BAGUA 1


In [112]:
distrito = distritos[nom_distrito]

In [113]:
def dist(cp1, cp2):
  x1, y1 = float(cp1['LATITUD']), float(cp1['LONGITUD'])
  x2, y2 = float(cp2['LATITUD']), float(cp2['LONGITUD'])

  return math.sqrt((x1 - x2)**2 + (y1 - y2)**2)

In [ ]:
G = nx.Graph()
col = 'CENTRO POBLADO'
for i, cp1 in distrito.iterrows():
  print(cp1[col])
  for j, cp2 in distrito.iterrows():
    if cp1[col] != cp2[col]:
      G.add_edge(cp1[col], cp2[col], weight=dist(cp1, cp2))

In [ ]:
gs.nx2gv(G, weighted=True, params={'size':'15'})

## Funcion calcularDistancia
Calcula la distancia entre 2 pares ordenados

In [116]:
def calcularDistancia(coords,pos1,pos2):
    x1,y1=coords[pos1]
    x2,y2=coords[pos2]
    return math.sqrt((x1 - x2)**2 + (y1 - y2)**2)


# Usando fuerza bruta


### Funcion buscarCaminoCorto
La variable temp guarda el valor de la suma de distancias
Se compara el valor de temp con el valor de solucion y se escoje el menor

In [117]:
def buscarCaminoCorto(combinaciones:list,coords:list,n):
    solution = 0

    temp = 0
    menor = 0
    for i in range(len(combinaciones)):
        temp = 0
        for j in range(0,n-1):
            temp+=calcularDistancia(coords,combinaciones[i][j],combinaciones[i][j+1])#calcula la distancia de cada posibilidad
            
        if i == 0:
            solution = temp
            menor = 0
        elif solution>temp:
            solution = temp
            menor = int(i)

    return solution,menor

### CODIGO
Se inserta los nombres de los pueblos y sus coordenadas.

Se usa la funcion zip para combinar la lista de "x" y "y" en formato de pares ordenados (x,y).


Se usa la funcion permutations() de itertool para hallar todas las combinaciones posibles de pueblos sin repetir.

In [118]:
orden = list()
def ordenar(ips:list):
  if len(orden) == 0:  #caso inicial
    orden = ips
  

In [172]:
def peru(distrito,order):
  #print(distrito) 
  #print(type(distrito))
  centrosPoblados =[]
  id = distrito.index[0]

  for i, cp1 in distrito.iterrows():
#  print(cp1[col])
    centrosPoblados.append(cp1[col])

  n = len(centrosPoblados)
  if n>8:
    print(distrito)
    print(n)
    sys.exit(0)
  #print(n)
#caminos validos
  pos = list(range(n))
  combinaciones = list(permutations(pos,n))

#coordenadas
  x=[]
  y=[]
  ids = []
  for i, cp1 in distrito.iterrows():
    ids.append(int(cp1['ID']))
    x.append(float(cp1['LATITUD']))
    y.append(float(cp1['LONGITUD']))


#poner coordenadas en formato (x,y)
  coordszip = zip(x,y)
  coords = list(coordszip)
  del x
  del y
  del coordszip
  solucion,menor = buscarCaminoCorto(combinaciones,coords,n)
  temp = combinaciones[menor]
  #ResponsePath = list()
  #print(temp)
  idOrden = list()
  for i in range(n):
    #ResponsePath.append({"cp":centrosPoblados[temp[i]],"lat":float(coords[temp[i]][0]),"lon":float(coords[temp[i]][1])})
    val = int(poblacionesDF.loc[id + temp[i],'order'])
    poblacionesDF.loc[id + temp[i],'order']=val+i+order
    idOrden.append(poblacionesDF.loc[id+temp[i],'ID'])
    #asignar el representante1 (ultimo) representante (primero)
    if poblacionesDF.loc[id + temp[i],'representante']==None:
      poblacionesDF.loc[id + temp[i],'representante']=id + temp[-1]
      poblacionesDF.loc[id + temp[i],'representante1']=id + temp[0]
      #siguiente
      if i != n-1: #ultimo aun no tiene siguiente
        poblacionesDF.loc[id+temp[i],'siguiente']=id+temp[i+1]
      #anterior
      if i != 0:
        poblacionesDF.loc[id+temp[i],'anterior']=id+temp[i-1]
    else:
      if i != n-1: #ultimo aun no tiene siguiente
        idTemp = poblacionesDF.loc[ids[temp[i+1]],'representante1']
        poblacionesDF.loc[ids[temp[i]],'siguiente'] = idTemp
        #poblacionesDF[idTemp,'anterior'] = ids[temp[i]]
      if i != 0:
        idTemp = poblacionesDF.loc[ids[temp[i-1]],'representante']
        poblacionesDF.loc[ids[temp[i]],'anterior']=idTemp
        #poblacionesDF[idTemp,'anterior'] = ids[temp[i]]
      poblacionesDF.loc[ids[temp[i]],'representante']=ids[temp[-1]]
      poblacionesDF.loc[ids[temp[i]],'representante1']=poblacionesDF.loc[ids[temp[0]],'representante1']
  #print(idOrden)
  last = poblacionesDF.loc[id+temp[-1]]
  #ordenar(idOrden)
  return last

  #print(json.dumps(ResponsePath))
 



In [179]:
url="https://raw.githubusercontent.com/lmcanavals/algorithmic_complexity/main/data/poblaciones.csv"
poblacionesDF = pd.read_csv(url)

poblacionesDF.loc[:,'order']=0
id = list(range(len(poblacionesDF)))
poblacionesDF.loc[:,'ID']=id
poblacionesDF.loc[:,'anterior']=None
poblacionesDF.loc[:,'siguiente']=None
poblacionesDF.loc[:,'representante']=None
poblacionesDF.loc[:,'representante1']=None
nomprovincias = poblacionesDF['PROVINCIA'].unique()

#----------------------------------------------------
nomdepartamentos = poblacionesDF['DEPARTAMENTO'].unique()
#print(nomdepartamentos)
departamento = dict()
lastDepartanento=list()
for dep in nomdepartamentos: #departamento
  departamento[dep] = poblacionesDF[poblacionesDF['DEPARTAMENTO']==dep]
  nomprovincias = departamento[dep]['PROVINCIA'].unique()
  #print(nomprovincias)
  lastProvincia = list()
  for prov in nomprovincias: #provincia
    distritos = provincias[prov]
    nomdistritos = provincias[prov]['DISTRITO'].unique()
    last = list()
    #-------------------------------------------------------------------------------------------------------------------------------------------
    #distrito
    for dist in nomdistritos: #distrito
      data = distritos[distritos['DISTRITO'] == dist]
      size = len(data)
      numdiv = size//5
      if numdiv == 0:
        #print("cp peque")
        last.append(peru(data,0))
      else: #dividir dataframe muy grandes
        numdiv+=1
        df_split = np.array_split(data,numdiv)
        lastTemp = list()
        for parte in range(len(df_split)):
          #print("cp dividido en",numdiv)
          lastTemp.append(peru(df_split[parte],0))
        #print(size,numdiv)
        lastIDs = list()
        #print(lastTemp[0][10])
        for elem in lastTemp:
          lastIDs.append(elem[9])
        #print(lastIDs)
        dftemp = poblacionesDF[poblacionesDF['ID'].isin(lastIDs)] #dataFrame para conectar centros poblados divididos
        #print("Centro poblado")
        #print(dftemp)
        #----------------------------------------------------------------------------------------------------------------------------------------#
        #distritos donde existen muchas diviciones
        size = len(dftemp)
        numdiv = size//5
        if numdiv == 0:
          #print(dftemp)
          last.append(peru(dftemp,0))
        else: #muchas divisiones
          numdiv+=1
          df_split = np.array_split(dftemp,numdiv)
          lastTemp = list()
          for parte in range(len(df_split)):
            #print("cp dividido en",numdiv)
            lastTemp.append(peru(df_split[parte],0))
          #print(size,numdiv)
          lastIDs = list()
          #print(lastTemp[0][10])
          for elem in lastTemp:
            lastIDs.append(elem[9])
          dftemp = poblacionesDF[poblacionesDF['ID'].isin(lastIDs)]
          last.append(peru(dftemp,0))
poblacionesDF.head(50)

SyntaxError: ignored

In [174]:
url="https://raw.githubusercontent.com/lmcanavals/algorithmic_complexity/main/data/poblaciones.csv"
poblacionesDF = pd.read_csv(url)

poblacionesDF.loc[:,'order']=0
id = list(range(len(poblacionesDF)))
poblacionesDF.loc[:,'ID']=id
poblacionesDF.loc[:,'anterior']=None
poblacionesDF.loc[:,'siguiente']=None
poblacionesDF.loc[:,'representante']=None
poblacionesDF.loc[:,'representante1']=None
nomprovincias = poblacionesDF['PROVINCIA'].unique()
#-------------------------------------------------------------------------
nomdepartamentos = poblacionesDF['DEPARTAMENTO'].unique()
#print(nomdepartamentos)
departamento = dict()
lastDepartanento=list()
for dep in nomdepartamentos: #departamento
  departamento[dep] = poblacionesDF[poblacionesDF['DEPARTAMENTO']==dep]
  nomprovincias = departamento[dep]['PROVINCIA'].unique()
  #print(nomprovincias)
  lastProvincia = list()
  for prov in nomprovincias: #provincia
    distritos = provincias[prov]
    nomdistritos = provincias[prov]['DISTRITO'].unique()
    last = list()
    #print("dist",nomdistritos)
    #print("prov",prov)
    #print("dep",dep)
    #-------------------------------------------------------------------------------------------------------------------------------------------
    #distrito
    for dist in nomdistritos: #distrito
      data = distritos[distritos['DISTRITO'] == dist]
      size = len(data)
      numdiv = size//5
      if numdiv == 0:
        #print("cp peque")
        last.append(peru(data,0))
      else: #dividir dataframe muy grandes
        numdiv+=1
        df_split = np.array_split(data,numdiv)
        lastTemp = list()
        for parte in range(len(df_split)):
          #print("cp dividido en",numdiv)
          lastTemp.append(peru(df_split[parte],0))
        #print(size,numdiv)
        lastIDs = list()
        #print(lastTemp[0][10])
        for elem in lastTemp:
          lastIDs.append(elem[9])
        dftemp = poblacionesDF[poblacionesDF['ID'].isin(lastIDs)] #dataFrame para conectar centros poblados divididos
        #print("Centro poblado")
        #print(dftemp)
        #----------------------------------------------------------------------------------------------------------------------------------------#
        #distritos donde existen muchas diviciones
        size = len(dftemp)
        numdiv = size//5
        if numdiv == 0:
          last.append(peru(dftemp,0))
        else: #muchas divisiones
          numdiv+=1
          df_split = np.array_split(dftemp,numdiv)
          lastTemp = list()
          for parte in range(len(df_split)):
            #print("cp dividido en",numdiv)
            lastTemp.append(peru(df_split[parte],0))
          #print(size,numdiv)
          lastIDs = list()
          #print(lastTemp[0][10])
          for elem in lastTemp:
            lastIDs.append(elem[9])
          dftemp = poblacionesDF[poblacionesDF['ID'].isin(lastIDs)]
          last.append(peru(dftemp,0))
        #------------------------------------------------------------------------------------------------------------------------------------------------
        
        #last.append(peru(lastTemp,10)) 
        #fin distrito
    #print(last)
    #-------------------------------------------------------------------------------------
    #provincias
    lastIDs = list()
    for elem in last:
      lastIDs.append(elem[9])
    dfDistrito = poblacionesDF[poblacionesDF['ID'].isin(lastIDs)] #dataframe para conectar todo el distrito #da el camino de provincia x
    size = len(dfDistrito)
    numdiv = size//5
    #print("dataset rpa cp")
    #print(dfDistrito)
    if numdiv == 0:
      #print("dis peque",numdiv,size)
      lastProvincia.append(peru(dfDistrito,100))
    else: #dividir dataframe muy grandes
      numdiv+=1
      df_split = np.array_split(dfDistrito,numdiv)
      lastTemp = list()
      for parte in range(len(df_split)):
        #print("dis dividido en",numdiv)
        lastTemp.append(peru(df_split[parte],100))
      lastIDs = list()
      for elem in lastTemp:
        lastIDs.append(elem[9])
      dftemp = poblacionesDF[poblacionesDF['ID'].isin(lastIDs)] #dataFrame para conectar distritos divididos
      #print("dis juntar")
      lastProvincia.append(peru(dftemp,100))
      #----------------------------------------------------------------------------------------------------
      #provincias con muchas diviciones
      size = len(dftemp)
      numdiv = size//5
      if numdiv == 0:
        last.append(peru(dftemp,10))
      else: #muchas divisiones
        numdiv+=1
        df_split = np.array_split(dftemp,numdiv)
        lastTemp = list()
        for parte in range(len(df_split)):
          #print("cp dividido en",numdiv)
          lastTemp.append(peru(df_split[parte],100))
        #print(size,numdiv)
        lastIDs = list()
        #print(lastTemp[0][10])
        for elem in lastTemp:
          lastIDs.append(elem[9])
        dftemp = poblacionesDF[poblacionesDF['ID'].isin(lastIDs)]
        lastProvincia.append(peru(dftemp,100))
        #---------------------------------------------------------------------

  lastIDs = list()
  #print(lastProvincia)

  for elem in lastProvincia:
    lastIDs.append(elem[9])
  dfProvincia = poblacionesDF[poblacionesDF['ID'].isin(lastIDs)] #dataframe para conectar todo el distrito #da el camino de provincia x
  size = len(dfProvincia)
  numdiv = size//5
  if numdiv == 0:
    #print("dis peque",numdiv,size)
    lastDepartanento.append(peru(dfProvincia,10000))
  else: #dividir dataframe muy grandes
    numdiv+=1
    df_split = np.array_split(dfProvincia,numdiv)
    lastTemp = list()
    #print("-----------------------------------------------------------")
    #print(dfProvincia)
    for parte in range(len(df_split)):
      #print("dep dividido en",numdiv,size)
      lastTemp.append(peru(df_split[parte],10000))
    lastIDs = list()
    for elem in lastTemp:
      lastIDs.append(elem[9])
    dftemp = poblacionesDF[poblacionesDF['ID'].isin(lastIDs)] #dataFrame para conectar distritos divididos
    #print("dis juntar")
    #lastDepartanento.append(peru(dftemp,1000))
    #----------------------------------------------------------------------------------------------------
    #departamentos con muchas diviciones
    size = len(dftemp)
    numdiv = size//5
    if numdiv == 0:
      lastDepartanento.append(peru(dftemp,1000))
    else: #muchas divisiones
      numdiv+=1
      df_split = np.array_split(dftemp,numdiv)
      lastTemp = list()
      for parte in range(len(df_split)):
        #print("cp dividido en",numdiv)
        lastTemp.append(peru(df_split[parte],1000))
      #print(size,numdiv)
      lastIDs = list()
      #print(lastTemp[0][10])
      for elem in lastTemp:
        lastIDs.append(elem[9])
      dftemp = poblacionesDF[poblacionesDF['ID'].isin(lastIDs)]
      lastDepartanento.append(peru(dftemp,1000))
      #---------------------------------------------------------------------


#print(departamento)
poblacionesDF.head(50)
#print(poblacionesDF)

,DEPARTAMENTO,PROVINCIA,DISTRITO,CENTRO POBLADO,LATITUD,LONGITUD,NMI1,NMI2,order,ID,anterior,siguiente,representante,representante1
0,AMAZONAS,BAGUA,ARAMANGO,EL MUYO (C.P.L.),-5.422545,-78.452990,311,1206,1,0,1,2,2,1
1,AMAZONAS,BAGUA,ARAMANGO,RESERVA UJUCANO,-5.427565,-78.444372,44,129,0,1,None,0,2,1
2,AMAZONAS,BAGUA,ARAMANGO,TUTUMBEROS,-5.327317,-78.444720,49,179,102,2,0,3,3,1
3,AMAZONAS,BAGUA,BAGUA,ALENYA BAJO,-5.682439,-78.495229,3,8,10103,3,15,14,15,12
4,AMAZONAS,BONGARA,CUISPES,MITOPAMPA,-5.938560,-77.965302,1,4,10101,4,None,9,9,4
5,AMAZONAS,BONGARA,JAZAN,NUEVA ESPERANZA,-5.935482,-77.976755,4,15,10101,5,15,1,15,12
6,AMAZONAS,BONGARA,JAZAN,PEDRO RUIZ GALLO,-5.941387,-77.977098,1365,5361,10005,6,7,8,8,5
7,AMAZONAS,BONGARA,JAZAN,PEDRO RUIZ,-5.940690,-77.976350,2,7,1,7,5,6,8,5
8,AMAZONAS,BONGARA,JAZAN,SAN JERONIMO,-5.950360,-77.978860,3,6,3,8,6,9,11,5
9,AMAZONAS,BONGARA,JAZAN,SANTA ROSA,-5.932149,-77.972108,1,5,1,9,9,10,9,4
